In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree



# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from category_encoders import TargetEncoder
from sklearn.ensemble import GradientBoostingRegressor



from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm


# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import os 
import sys
sys.path.append("../")
from src import soporte_ajuste as sa

pd.options.display.float_format = "{:,.2f}".format 
pd.set_option("display.max_columns", None)

In [4]:
df=pd.read_csv("datos/datos_coding.csv", index_col=0).reset_index(drop=True)

In [8]:
#separamos x e y para predecir nuestro modelo
x= df.drop(columns="price")
y= df[["price"]]

In [9]:
#Separamos train y test
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size= 0.7, random_state=42)

Modelo

In [10]:


# Si y es un DataFrame, conviértelo a Serie para facilitar el procesamiento
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]  # Convertir a Serie usando la primera columna

# Dividimos los datos en conjuntos de entrenamiento (train) y prueba (test)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.8, random_state=42
)

# Verificar las formas de los conjuntos de datos resultantes
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de y_test: {y_test.shape}")

# Definir los hiperparámetros a evaluar
params_arbol = {
    "max_depth": [5, 7, 9, 10, 11],
    "min_samples_split": [10, 50, 100, 200],
    "min_samples_leaf": [10, 50, 100, 200],
    "max_leaf_nodes": [4, 6, 8, 10, 20, 30]
}

# Crear el modelo base
dt_model = DecisionTreeRegressor(random_state=42)

# Configurar la búsqueda de los mejores hiperparámetros
grid_search = GridSearchCV(
    estimator=dt_model,
    param_grid=params_arbol,
    scoring="neg_mean_squared_error",  # Usar MSE para optimizar
    cv=5,  # Validación cruzada con 5 folds
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=1
)

# Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_dt_model = grid_search.best_estimator_
print(f"Mejores parámetros: {grid_search.best_params_}")

# Hacer predicciones con el mejor modelo
y_pred_dt = best_dt_model.predict(X_test)

# Calcular el RMSE
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))

# Calcular el coeficiente R^2
r2_dt = r2_score(y_test, y_pred_dt)

# Mostrar resultados
print("Precio real vs Predicción:")
display(pd.DataFrame({'Real': y_test.values, 'Predicción': y_pred_dt}).head(2))  # Mostrar las primeras 2 filas
print(f"RMSE del Árbol de Decisión: {rmse_dt:.2f}")
print(f"R^2 del Árbol de Decisión: {r2_dt:.2f}")


Forma de X_train: (70, 24)
Forma de X_test: (283, 24)
Forma de y_train: (70,)
Forma de y_test: (283,)
Fitting 5 folds for each of 480 candidates, totalling 2400 fits
Mejores parámetros: {'max_depth': 5, 'max_leaf_nodes': 4, 'min_samples_leaf': 10, 'min_samples_split': 10}
Precio real vs Predicción:


,Real,Predicción
0,720.00,725.56
1,699.00,687.56


RMSE del Árbol de Decisión: 27.50
R^2 del Árbol de Decisión: 0.80
